### $\S3$. 共享变量

#### 一. 引言
1. 编写正确的多线程程序, 关键在于对共享变量的处理
2. 第2章介绍了使用"同步"来避免在同一时刻访问相同的变量;本章将介绍如何共享和发布对象, 使他们能够安全的有多个线程同时访问
3. synchronized关键字的一个常见误解是:   
    (1) 它只能用于实现共享变量一系列操作的原子性, 或是将修饰的变量作为一种"临界区";   
    (2) 而它还有另一个重要的作用: 变量的内存可见性; 如同"同步"的字面意思一样, 它使得一个线程在修改了共享变量的状态后, 其他线程能够看到这个对象变化后的状态. 
  
#### 二. 可见性
1. 非原子性的64位操作  
    Java内存模型要求, 变量的读取和写入操作都必须是原子性的. 但对于非volatile修饰的64位数值型变量,会拆成2个32位的读写操作, 使得64位数值型变量的读取和写入变成2个异步的32位原子操作. 如果2个线程分别在读写这个变量, 那么很可能读取到某个值的高32位和另一个值的低32位.
2. 加锁与可见性  
    加锁的含义不仅局限于互斥行为, 还包括内存可见性. 为了确保所有线程都能看到共享变量的最新值, 所有执行读操作和写操作的线程都必须在同一个锁上进行同步. (锁中包含代码块内的共享变量)
3. volatile关键字  
    (1) Java语言提供了一种稍弱的同步机制, 用来确保变量的修改可以通知其他持有这个变量引用的线程.  
    (2) 当变量被修饰成volatile的后, 这这个变量不会被缓存在寄存器等其它处理器不可见的地方, 也不会将该变量上的操作和其它内存操作一起进行重排序, 因此, 读取volatile类型的变量总会返回最新写入的值  
    (3) volatile只提供变量的内存可见性, 而不具备synchronized的"临界区"特性或是从操作的原子性特征, 因此也不会产生死锁, 但功能稍弱.  
    (4) volatile的典型用法:  
       修饰某个标记性变量(true/false), 但注意, volatile并不能保证递增操作的原子性(count++)
        
#### 三. 发布与溢出
1. 发布:   
    "发布"是指对象在当前作用于之外的代码使用, 例如, 讲一个指向该对象的引用保存到其它代码可以访问的地方, 或是在非private的方法中返回此对象
2. 逃逸:
    "逃逸"是指, 某个不应该被发布的对象被发布. 例如:  
    (1) getter中返回了某个私有变量, 在多线程操作下, 不进行同步会产生逃逸  
    (2) this逃逸:   
     a. 如果在构造函数中, 启动了一个新线程, 且这个线程使用了对象的其他属性和方法, 则此时产生this对象逃逸, 因为对象还未构造完毕就被发布出去.  
     b. 如果想在构造函数中启动一个线程, 有2个解决办法:  
       (a) 构造函数中只负责创建线程, 但不启动这个线程; 而是通过一个start/initial方法来启动  
       (b) 也可以使用一个私有的构造器和一个静态的工厂方法, 从而避免错误的发布
     ```java
    // this逃逸
     public class ThisEscape {
        public ThisEscape(EventSource source) {
            source.registerListener(new EventListener() {  // registerListener方法会启动一个线程
              public void onEvent(Event e) {
                this.doSomething(e);    // 调用了this.doSomething, 但此时构造函数还未走完, 对象还未创建完毕
              }
            });
           }
        public void doSomething(Event e) {}
     }
     ```
    this逃逸的改进如下
     ```java
    public class SafeListener {
      private final EventListener listener;
      // 私有的构造器
      private SafeListener() {
        listener = new EventListener() {
          public void onEvent(Event e) {
            this.doSomething(e);  // 仍然使用this,doSomething, 但register操作在静态方法中
          }
        };
      }
      // 静态的工厂方法
      public static SafeListener newInstance(EventSource source) {
        SafeListener safe = new SafeListener();  // 对象构造完毕, 正确发布
        source.registerListener(safe.listener);  // 另一个线程正常使用
        return safe;
      }

      void doSomething(Event e) {
      }
    }
     ```
     
#### 四. 不变形
1. 满足同步需求的另一种方法是使用不可变对象. 不可变对象的状态不嗯呢该发生变化, 也就没有了多线程共享变量的一切问题  
2. 即使一个对象的所有属性都是final的, 这个对象仍然可能不是不可变的; 因为final类型的变量虽然不能执行重新复制, 但其引用的对象却可以发生变化, 如下操作是被允许的:  
    ```java
    class A{
    public String name;
    public A(String name) {
        this.name = name;
        }
    }
    public class Test {
        public final A a;

        public Test(A a) {
            this.a = a;
        }

        public static void main(String[] args) {
            Test haha = new Test(new A("haha"));
            System.out.println(haha.a.name);  // haha
            haha.a.name = "hehe";
            System.out.println(haha.a.name);  // final类型的a, 其name已经变成"hehe"
        }
    }
    ```
    
#### 五. 如何正确的发布对象
1. 常用的安全发布对象的方式有以下几种:  
    (1) 在静态工厂方法中, 初始化一个对象的引用  
    (2) 将对象的引用保存到volatile作用域或是AtomicReferance对象中   
    (3) 将对象的应用, 宝尊子啊某个已被正确构造的对象的final作用域中  
    (4) 将对象的引用用锁保护起来  

### \[附录\]: synchrinized的几种用法
#### 一. 类级别的锁  (所有同步针对该类的所有对象)
1. 同步静态方法   
同步静态方法是类级别的锁，一旦任何一个线程进入这个方法，其他所有线程将无法访问这个类的任何同步类锁的方法。
    ```java
    public synchronized static void fun() { }
    ```
2. 同步代码块锁类  
下面提供了两种同步类的方法，锁住效果和同步静态方法一样，都是类级别的锁，同时只有一个线程能访问带有同步类锁的方法。
    ```java
    private void fun() {
        synchronized (this.getClass()) { }
    }
    ```
    
#### 二. 对象计别的锁 (所有同步只针对同一个对象)
1. 同步普通方法
    ```java
    public synchronized void fun() { }
    ```
2. 同步代码块中使用this对象/其它对象作为锁
    ```java
    public void fun() {
        synchronized (this) { }
    }

    public void fun() {
        synchronized (LOCK) { }
    }
    ```